# 10.APPLY NODE2VEC ON Ingredient-Recipe Biparite Graph

# SAVE THE MODEL AND EMBEDDINGS FOR FINAL RECOMMENDATION

In [1]:
import os
import pandas as pd
irdata = pd.read_csv('no_drop.csv')
irdata.head()

,recipe_id,recipe_name,image_url,ingredients,cooking_directions,nutritions
0,240488,"Pork Loin, Apples, and Sauerkraut",https://images.media-allrecipes.com/userphotos...,"sauerkraut,granny smith apples,onion,caraway s...",{'directions': u'Prep\n15 m\nCook\n2 h 30 m\nR...,"{u'niacin': {u'hasCompleteData': False, u'name..."
1,218939,Foolproof Rosemary Chicken Wings,https://images.media-allrecipes.com/userphotos...,"chicken wings,sprigs,head garlic,oil,pepper,salt","{'directions': u""Prep\n20 m\nCook\n40 m\nReady...","{u'niacin': {u'hasCompleteData': True, u'name'..."
2,87211,Chicken Pesto Paninis,https://images.media-allrecipes.com/userphotos...,"bread,basil pesto,chicken,bell pepper,onion,mo...",{'directions': u'Prep\n15 m\nCook\n5 m\nReady ...,"{u'niacin': {u'hasCompleteData': True, u'name'..."
3,245714,Potato Bacon Pizza,https://images.media-allrecipes.com/userphotos...,"potatoes,strips bacon,sauce,whipping cream,but...",{'directions': u'Prep\n20 m\nCook\n45 m\nReady...,"{u'niacin': {u'hasCompleteData': True, u'name'..."
4,218545,Latin-Inspired Spicy Cream Chicken Stew,https://images.media-allrecipes.com/userphotos...,"skinless boneless breast halves,tomatoes,salsa...",{'directions': u'Prep\n10 m\nCook\n8 h 15 m\nR...,"{u'niacin': {u'hasCompleteData': False, u'name..."


In [2]:
uni=pd.read_csv('Updated_Unique_Ingredients.csv')
uni.head()

,Unique_Ingredients
0,candlenuts
1,hillshire farm smoked sausage
2,zest peel
3,marsala wine optional
4,asparagus liquid


In [3]:
irdata['recipe_id'] = 'r' + irdata['recipe_id'].astype(str)

In [4]:
irdata.head()

,Unnamed: 0,recipe_id,recipe_name,image_url,ingredients,cooking_directions,nutritions
0,0,r240488,"Pork Loin, Apples, and Sauerkraut",https://images.media-allrecipes.com/userphotos...,"sauerkraut,granny smith apples,onion,caraway s...",{'directions': u'Prep\n15 m\nCook\n2 h 30 m\nR...,"{u'niacin': {u'hasCompleteData': False, u'name..."
1,1,r218939,Foolproof Rosemary Chicken Wings,https://images.media-allrecipes.com/userphotos...,"chicken wings,sprigs,head garlic,oil,pepper,salt","{'directions': u""Prep\n20 m\nCook\n40 m\nReady...","{u'niacin': {u'hasCompleteData': True, u'name'..."
2,2,r87211,Chicken Pesto Paninis,https://images.media-allrecipes.com/userphotos...,"bread,basil pesto,chicken,bell pepper,onion,mo...",{'directions': u'Prep\n15 m\nCook\n5 m\nReady ...,"{u'niacin': {u'hasCompleteData': True, u'name'..."
3,3,r245714,Potato Bacon Pizza,https://images.media-allrecipes.com/userphotos...,"potatoes,strips bacon,sauce,whipping cream,but...",{'directions': u'Prep\n20 m\nCook\n45 m\nReady...,"{u'niacin': {u'hasCompleteData': True, u'name'..."
4,4,r218545,Latin-Inspired Spicy Cream Chicken Stew,https://images.media-allrecipes.com/userphotos...,"skinless boneless breast halves,tomatoes,salsa...",{'directions': u'Prep\n10 m\nCook\n8 h 15 m\nR...,"{u'niacin': {u'hasCompleteData': False, u'name..."


In [5]:
recipe_id=list(irdata.recipe_id)

In [6]:
recipe_id[:5]

['r240488', 'r218939', 'r87211', 'r245714', 'r218545']

In [7]:
unique_ingredients=list(uni.Unique_Ingredients)

In [8]:
unique_ingredients[:5]

['candlenuts',
 'hillshire farm smoked sausage',
 'zest peel',
 'marsala wine optional',
 'asparagus liquid']

In [9]:
total_ingredients=list(irdata.ingredients)

In [10]:
import networkx as nx
import matplotlib.pyplot as plt
 
#Create a graph
G = nx.DiGraph()

#Add nodes
G.add_nodes_from(unique_ingredients, bipartite=0)
G.add_nodes_from(recipe_id, bipartite=1)

# Iterating through Unique_ingredients to comapre with each recipe's ingredients
for i in range(len(recipe_id)):
    for x in unique_ingredients:
        if x in total_ingredients[i]:
            G.add_edge(x,recipe_id[i])

In [11]:
import networkx as nx
from node2vec import Node2Vec

In [12]:
node2vec = Node2Vec(G, dimensions=128, walk_length=80, num_walks=10, workers=1)

Generating walks (CPU: 1):   0%|                                                                | 0/10 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|███████████████████████████████████████████████████████| 10/10 [00:09<00:00,  1.02it/s]


In [13]:
model = node2vec.fit(window=10, min_count=1, batch_words=4)

In [14]:
model.wv.save_word2vec_format("embeddings_ingredients_recipe")

In [16]:
model.save("model_ingredients_recipe")

In [2]:
import gensim

# Load pre-trained Word2Vec model.
model = gensim.models.Word2Vec.load("model_ingredients_recipe")

# The inbulit module most_similar uses KNN along with cosine similarity as a metric, as gives the similar nodes to the input given  

In [13]:
for node, _ in model.most_similar('butterhead boston',topn=10):
    # Show only ingredient
    if any(char.isdigit() for char in node) == False:
        print(node)

brown pistolette rolls
pound chicken
links sausage eckrich
shrimp brine
dogs slices
fluid ounce jagermeister liqueur
sugar sugar substitute
prosciutto thin
dole classic coleslaw


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [9]:
for node, _ in model.most_similar('r17525'):
    # Show only recipes
    if any(char.isdigit() for char in node) == True:
        print(node)

r213807
r8809
r95817
r212476
r127325
r23622
r87211
r235652


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [12]:
for node, _ in model.most_similar('reducedfat reducedsodium cream chicken soup campbells healthy request',topn=30):
    # Show only ingredient
    if any(char.isdigit() for char in node) == False:
        print(node)

pomegranate cranberry dressing
family teabags orange pekoe tea
racks st louis style ribs
garlic bread
bunch cilantro leaves
cucumber sauce
head endive


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [16]:
for node, _ in model.most_similar('zest peel',topn=10):
    # Show only ingredient
    if any(char.isdigit() for char in node) == False:
        print(node)

noodles substitute linguine
basil pesto classico ™
molasses
citrus corn salad
baby potatoes


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [22]:
similar_ingredients=list()
for node, _ in model.most_similar('butterhead boston',topn=10):
    # Show only ingredient
    if any(char.isdigit() for char in node) == False:
        similar_ingredients.append(node)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


In [23]:
print(similar_ingredients)

['brown pistolette rolls', 'pound chicken', 'links sausage eckrich', 'shrimp brine', 'dogs slices', 'fluid ounce jagermeister liqueur', 'sugar sugar substitute', 'prosciutto thin', 'dole classic coleslaw', 'old el paso enchilada sauce']


In [28]:
for node, _ in model.most_similar(similar_ingredients,topn=10):
    # Show only ingredient
    if any(char.isdigit() for char in node) == True:
        print(node)

r149591
r16800
r229977
r89845
r216922
r55049
r14168
r86037
r23082
r228485


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [8]:
for node, k in model.most_similar('chicken broth',topn=60):
    # Show only ingredient
    if any(char.isdigit() for char in node) == False:
        print(node)

dish pie crusts
coarse bulgur
beef sausage hillshire farm
mango
paste kochujang
asparagus ends
wedges lime
chocolate mix
peach brandy


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [3]:
for node, k in model.most_similar('bacon',topn=60):
    # Show only ingredient
    if any(char.isdigit() for char in node) == False:
        print(node)

head cabbage cut eighths
cream cheese chives
salt spice islands beau monde
ginger crust
banana pepper
ramen noodles
bacon pieces
poultry bells
fluid ounce scotch whiskey
artichokes heart
chihuaua cheese
spinach pasta


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [4]:
for node, k in model.most_similar('turkey',topn=60):
    # Show only ingredient
    if any(char.isdigit() for char in node) == False:
        print(node)

bars caramelpeanut nougat candy
kielbasa sausage
coarsegrain mustard
dole carrots
oatmeal mccanns
friendship bread starter
roll dough
clementines
fluid ounce maple syrup
round pepper
juice pepper rings
boneless pork sirloin roast kitchen twine
pint ripe blackberries room temperature
sauce bolognese


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [5]:
for node, k in model.most_similar('mustard',topn=60):
    # Show only ingredient
    if any(char.isdigit() for char in node) == False:
        print(node)

envelope reducedsodium taco mix
strawberry syrup
quarts buttermilk
chopped
cooking oil
flaxseed milk
garlic herb seasoning
conversation heart candies necco
avocado lengthwise pieces
fluid ounce kirschwasser
kirschwasser optional
chocolate milk balls
proof grain alcohol everclear™
lamb loin thick
tomatoes onion


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [6]:
for node, k in model.most_similar('quarts water',topn=60):
    # Show only ingredient
    if any(char.isdigit() for char in node) == False:
        print(node)

lobster mushrooms cubes
grain rice mix herbs
bottle pimento
acai berry pulp frozen
zucchini cut strips peeler
crisco original nostick cooking spray
hickory wood chips
twist lemon garnish
sugar snap peas
capers dry
shoulder chops
fluid ounce cola
skinless boneless strips
barbeque sauce stubbs
milk lukewarm
plain tea pieces
souvlaki marinade
coconut extract


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
